# Wrapper notebook for submitting the RBP maps script to TSCC

In [1]:
import pandas as pd
import os
import json
import yaml
import glob
# import rethinkdb as r
from collections import defaultdict
from qtools import Submitter
from encode import manifest_helpers as m

from tqdm import tnrange, tqdm_notebook
pd.set_option("display.max_colwidth", 10000)


# Define manifests, directories, etc.
- SBDS-BGKLV24-K562 and PPIL4-BGKLV24-K562 were changed in the K562.csv list from SBDS-BGKLV24_2-K562 and PPIL4-BGKLV24_2-K562 per an email discussion from xintao.

In [2]:
current_date = '11-27-2018'
clip_manifest = '/projects/ps-yeolab3/bay001/reference_data/ENCODE/ENCODE_FINAL_ANNOTATIONS.uidsonly.txt.manifest.txt'
density_runner = '/home/bay001/projects/codebase/rbp-maps/maps/plot_map.py'
ri_annotation_dir = '/projects/ps-yeolab3/bay001/maps/current_annotations/ri_renamed'
control_dir = '/projects/ps-yeolab3/bay001/maps/current_annotations/erics_controls/nonredundant_renamed/ri/'

clip_df = pd.read_table(clip_manifest)

master_table = pd.read_table(
    '/projects/ps-yeolab3/encode/rnaseq/eCLIP_finalstatus_20180406_ENCODE_combined_RNASEQ.tsv'
)
del master_table['eCLIP_Antibody_Lot_#']  # there is a NaN value in here that's screwing things up. Don't care about antibody anyway, so delete
merged = pd.merge(
    clip_df,
    master_table,
    how='right',
    left_on=['uID'],
    right_on=['eCLIP_Final_internal_accession']
)
merged.dropna(subset=['eCLIP_Final_internal_accession','SE_jxc_file'], inplace=True)
print(merged.shape)

(203, 43)


In [3]:
def get_clip_file_from_uid(uid, df=merged):
    df = df[df['uID']==uid]
    return df['CLIP_rep1'].values[0], \
        df['CLIP_rep2'].values[0], \
        df['INPUT'].values[0], \
        df['eCLIP_Official_Gene_Symbol'].values[0], \
        df['eCLIP_Cell_Line'].values[0], \
        df['SE_jxc_file'].values[0]
        
r1, r2, i, rbp, cell, jxc_se  = get_clip_file_from_uid('204')

def get_ri_annotations_from_jxc_se(jxc, jxc_dir=ri_annotation_dir, event='ri'):
    """ jxc contains the basename of the junction counts file """
    orig_file = os.path.join(jxc_dir, jxc)
    positive = orig_file.replace('.SE.MATS.JunctionCountOnly.txt','-included-upon-knockdown'.format(event.upper()))
    negative = orig_file.replace('.SE.MATS.JunctionCountOnly.txt','-excluded-upon-knockdown'.format(event.upper()))
    if not os.path.exists(positive):
        positive = None
    if not os.path.exists(negative):
        negative = None
    return positive, negative

get_ri_annotations_from_jxc_se(jxc_se)

('/projects/ps-yeolab3/bay001/maps/current_annotations/ri_renamed/RBFOX2-BGHLV26-HepG2.set26-included-upon-knockdown',
 '/projects/ps-yeolab3/bay001/maps/current_annotations/ri_renamed/RBFOX2-BGHLV26-HepG2.set26-excluded-upon-knockdown')

# Plot RI

In [4]:
clip_df = pd.read_table(clip_manifest)

events_to_annotation_dict = {
    'ri':'/projects/ps-yeolab3/bay001/maps/current_annotations/ri_renamed/'
}


img_extensions = ['png']
# out_base = '/projects/ps-yeolab3/bay001/maps/current/'
output_dir = '/projects/ps-yeolab3/bay001/maps/current/ri/'
pos_splicing_suffix = '-included-upon-knockdown' # positive RMATS -> more intron inclusion (upon knockdown)
neg_splicing_suffix = '-excluded-upon-knockdown' # negative RMATS -> more intron splicing (upon knockdown)

for img_extension in img_extensions:
    for event, annotation_dir in events_to_annotation_dict.iteritems():
        no_rnaseq = [] # uIDs for which we don't have rna seq expt ids for
        no_rnaseq_yet = [] # uIDs for which we have an expt id, but haven't downloaded the data yet
        cmds = []
        for uid in merged['uID']:
            r1, r2, i, rbp, cell, jxc_se = get_clip_file_from_uid(uid)
            if cell == 'HepG2':
                background1 = os.path.join(annotation_dir, 'HepG2-constitutive-introns')
                background2 = os.path.join(annotation_dir, 'HepG2-all-retained-introns')
                background3 = os.path.join(annotation_dir, 'HepG2-greater-than-50-percent-spliced')
                background4 = os.path.join(annotation_dir, 'HepG2-greater-than-50-percent-retained')
            elif cell == 'K562':
                background1 = os.path.join(annotation_dir, 'K562-constitutive-introns')
                background2 = os.path.join(annotation_dir, 'K562-all-retained-introns')
                background3 = os.path.join(annotation_dir, 'K562-greater-than-50-percent-spliced')
                background4 = os.path.join(annotation_dir, 'K562-greater-than-50-percent-retained')
            else:
                print(cell)
            positive, negative = get_ri_annotations_from_jxc_se(jxc_se)
            ### OLD DEPRECATED ###
            # splicing_prefix = m.get_rnaseq_splicing_prefix_from_rbpname(rnaseq_manifests, rbp, cell)
            # if(splicing_prefix == "NO_RNASEQ"): # we don't have an rna seq expt for this clip:
            #     no_rnaseq.append(uid)
            # else:
            # positive, negative = m.get_annotations_from_splicing_prefix(
            #     annotation_dir, 
            #     splicing_prefix,
            #     pos_splicing_suffix=pos_splicing_suffix,
            #     neg_splicing_suffix=neg_splicing_suffix
            # )
            ######################
            if positive is not None and negative is not None:
                pos_prefix = os.path.basename(positive).split('-')[0]
                neg_prefix = os.path.basename(negative).split('-')[0]
                if not (pos_prefix in rbp and neg_prefix in rbp):
                    print(
                        'warning, these dont match: {}, {}, {}'.format(
                            rbp, 
                            os.path.basename(positive),
                            os.path.basename(negative)
                        )
                    )
                for r in [r1, r2]:
                    name = os.path.basename(r).replace('.bam','.{}'.format(img_extension))
                    output_filename = os.path.join(
                        output_dir,
                        name
                    )
                    cmd = "python " + density_runner
                    cmd = cmd + " --event {}".format(event)
                    cmd = cmd + " --ipbam {}".format(r)
                    cmd = cmd + " --inputbam {}".format(i)
                    cmd = cmd + " --output {}".format(output_filename)
                    if positive is not None and negative is not None:
                        cmd = cmd + " --annotations {} {} {} {} {} {}".format(
                            positive, negative, background1, background2, background3, background4
                        )
                        cmd = cmd + " --annotation_type {} {} {} {} {} {}".format(
                            'rmats', 'rmats', 'eric', 'eric', 'eric', 'eric', 
                        )
                    cmd = cmd + " --testnum {} {}".format(0, 1)
                    cmd = cmd + " --bgnum {}".format(3)
                    cmd = cmd + " --sigtest mannwhitneyu"
                    cmds.append(cmd)
    Submitter(
        cmds, 
        "{}_NR_{}".format(event, img_extension), 
        sh='/projects/ps-yeolab3/bay001/maps/bash_scripts/{}_NR_{}.sh'.format(
            event, img_extension
        ),
        submit=False,
        array=True,
        queue='home-yeo',
        walltime='2:00:00'
    )
print("{} uIDs for which we don't have splicing data for: {}".format(event, len(no_rnaseq)))
print("{} uIDs for which we have an RNASEQ ID, but we don't have the splicing data yet: {}".format(event, len(no_rnaseq_yet)))

warning, these dont match: DROSHA, RNASEN50-BGKLV28-K562.set28-included-upon-knockdown, RNASEN50-BGKLV28-K562.set28-excluded-upon-knockdown
warning, these dont match: YBX3, CSDA-BGKLV24-K562.set24-included-upon-knockdown, CSDA-BGKLV24-K562.set24-excluded-upon-knockdown
warning, these dont match: YBX3, CSDA-BGHLV20-HepG2.set20-included-upon-knockdown, CSDA-BGHLV20-HepG2.set20-excluded-upon-knockdown
ri uIDs for which we don't have splicing data for: 0
ri uIDs for which we have an RNASEQ ID, but we don't have the splicing data yet: 0


Writing 364 tasks as an array-job.
Wrote commands to /projects/ps-yeolab3/bay001/maps/bash_scripts/ri_NR_png.sh.


In [5]:
peak_dir = '/projects/ps-yeolab3/bay001/maps/current_annotations/se_peak_idr_bigbeds'

img_extensions = ['svg']
out_base = '/projects/ps-yeolab3/bay001/maps/current/'
norm_method = 0
event = 'ri'
annotation_dir = ri_annotation_dir

for img_extension in img_extensions: # for each image extension
    no_rnaseq = [] # uIDs for which we don't have rna seq expt ids for
    no_rnaseq_yet = [] # uIDs for which we have an expt id, but haven't downloaded the data yet
    cmds = []
    sf_cmds = []
    output_dir = os.path.join(out_base, '{}_IDR'.format(event))
    for uid in merged['uID']:
        r1, r2, i, rbp, cell, jxc_se = get_clip_file_from_uid(uid, merged)

        if cell == 'HepG2':
            background1 = os.path.join(annotation_dir, 'HepG2-constitutive-introns')
            background2 = os.path.join(annotation_dir, 'HepG2-all-retained-introns')
            background3 = os.path.join(annotation_dir, 'HepG2-greater-than-50-percent-spliced')
            background4 = os.path.join(annotation_dir, 'HepG2-greater-than-50-percent-retained')
        elif cell == 'K562':
            background1 = os.path.join(annotation_dir, 'K562-constitutive-introns')
            background2 = os.path.join(annotation_dir, 'K562-all-retained-introns')
            background3 = os.path.join(annotation_dir, 'K562-greater-than-50-percent-spliced')
            background4 = os.path.join(annotation_dir, 'K562-greater-than-50-percent-retained')
        else:
            print(cell)


        positive, negative = get_ri_annotations_from_jxc_se(
            jxc_se, annotation_dir, event
        )
        if(positive == None or negative == None):
            no_rnaseq_yet.append(uid)
        else:
            if not (rbp in positive and rbp in negative):
                print(
                    'warning, these dont match: {}, {}, {}'.format(
                        rbp, 
                        os.path.basename(positive),
                        os.path.basename(negative)
                    )
                )
            pos_prefix = os.path.basename(positive).split('-')[0]
            neg_prefix = os.path.basename(negative).split('-')[0]
            if not (pos_prefix in rbp and neg_prefix in rbp):
                print(
                    'warning, these dont match: {}, {}, {}'.format(
                        rbp, 
                        os.path.basename(positive),
                        os.path.basename(negative)
                    )
                )
            peak_files = glob.glob(os.path.join(
                peak_dir, 
                '{}.01v02.IDR.out.0102merged.bed.blacklist_removed.bed.p0f0.bed.sorted.bed.bb'.format(
                    uid
                )
            ))

            assert len(peak_files) == 1
            for peak in peak_files:
                output_filename = os.path.join(
                    output_dir, 
                    os.path.basename(peak).replace(
                        '.p0f0.bed.sorted.bed.bb', '.{}'.format(img_extension)
                    )
                )

                cmd = "module load rbpmaps;python " + density_runner
                cmd = cmd + " --event {}".format(event)
                cmd = cmd + " --peak {}".format(peak)
                cmd = cmd + " --output {}".format(output_filename)
                # if positive is not None and negative is not None:
                #     cmd = cmd + " --annotations {} {} {} {} {} {}".format(
                #         positive, negative, background1, background2, background3, background4
                #     )
                #     cmd = cmd + " --annotation_type {} {} {} {} {} {}".format(
                #         'rmats', 'rmats', 'eric', 'eric', 'eric', 'eric'
                #     )
                if positive is not None and negative is not None:
                    cmd = cmd + " --annotations {} {}".format(
                        positive, negative
                    )
                    cmd = cmd + " --annotation_type {} {}".format(
                        'rmats', 'rmats'
                    )
                cmd = cmd + " --bgnum {}".format(1)
                cmd = cmd + " --testnum {}".format(0)
                cmd = cmd + " --normalization {}".format(norm_method)
                cmd = cmd + " --sigtest {}".format('fisher')
                if not os.path.exists(output_filename):
                    cmds.append(cmd)
    bash_script_sh = '/projects/ps-yeolab3/bay001/maps/bash_scripts/{}/{}_NR_{}.sh'.format(
        current_date, event, img_extension
    )
    Submitter(
        cmds, 
        "{}_NR_{}".format(event, img_extension), 
        sh=bash_script_sh,
        submit=True,
        array=True,
        walltime='2:00:00',
        queue='home-yeo'
    )
    with open(bash_script_sh.replace('.sh','.missing.txt'), 'w') as o:
        for no in no_rnaseq:

            o.write(
                '{}\t{}\n'.format(
                    get_clip_file_from_uid(uid=no, df=merged)[3],
                    get_clip_file_from_uid(uid=no, df=merged)[4],
                )
            )
        print("\n\nNO SUFFICIENT POSITIVE OR NEGATIVE SIGNIFICANT ANNOTATIONS:")
        for no in no_rnaseq_yet:
            print(get_clip_file_from_uid(uid=no, df=merged)[3]),

warning, these dont match: DROSHA, RNASEN50-BGKLV28-K562.set28-included-upon-knockdown, RNASEN50-BGKLV28-K562.set28-excluded-upon-knockdown
warning, these dont match: DROSHA, RNASEN50-BGKLV28-K562.set28-included-upon-knockdown, RNASEN50-BGKLV28-K562.set28-excluded-upon-knockdown
warning, these dont match: YBX3, CSDA-BGKLV24-K562.set24-included-upon-knockdown, CSDA-BGKLV24-K562.set24-excluded-upon-knockdown
warning, these dont match: YBX3, CSDA-BGKLV24-K562.set24-included-upon-knockdown, CSDA-BGKLV24-K562.set24-excluded-upon-knockdown
warning, these dont match: YBX3, CSDA-BGHLV20-HepG2.set20-included-upon-knockdown, CSDA-BGHLV20-HepG2.set20-excluded-upon-knockdown
warning, these dont match: YBX3, CSDA-BGHLV20-HepG2.set20-included-upon-knockdown, CSDA-BGHLV20-HepG2.set20-excluded-upon-knockdown


Writing 179 tasks as an array-job.
Wrote commands to /projects/ps-yeolab3/bay001/maps/bash_scripts/11-27-2018/ri_NR_svg.sh.




NO SUFFICIENT POSITIVE OR NEGATIVE SIGNIFICANT ANNOTATIONS:
FKBP4 SLTM FAM120A CSTF2T SND1 ILF3 SMNDC1 DDX3X KHDRBS1 BUD13 CSTF2 DDX6 DDX51 LSM11 DHX30 DDX52 FUS NIP7 EXOSC5 SDAD1 SUB1


Submitted script to queue home-yeo.
 Job ID: 13977654
